In [136]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [137]:
old_data = pd.read_excel('report_invest_30.09.2024.xlsx', sheet_name='snapshot')
old_data.shape

(104629, 25)

In [13]:
new_data = pd.read_excel('automated_for_test_30_09_2024.xlsx')
new_data.shape

c:\Users\KlimovaAnnaA\AppData\Local\miniconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(12078, 39)

In [62]:
ccy_dict = {'Рубль':'RUB','Евро':'EUR', 'Долла':'USD', 'Рупия':'INR', 'Тенге':'KZT'}
old_data.currency = old_data.currency.replace(CCY_tech_dict)
new_data.currency = new_data.currency.replace(ccy_dict)

In [131]:
# old_data_columns_list = ['companyCode', 'companyName', 'plantId', 'plantName','sector', 'sectorName', 'wbsElement', 'wbsElementName', 'vendorId','vendorName', 'projectTypeCode', 'projectTypeName', 'projectProfile','projectId', 'projectName', 'costElement', 'costElementName','versionCode', 'versionName', 'calendarYearMonth', 'currency', 'Сумма ПВК']
old_data_columns_list = ['companyCode', 'plantId', 'sector', 'wbsElement', 'vendorId', 'versionCode', 'calendarYearMonth']
# old_data_work = old_data[old_data_columns_list]
# to_remove = ['wbsElement','wbsElementName','projectTypeName','costElement','Сумма ПВК']
to_count = 'Сумма ПВК'
# for x in to_remove:
#     old_data_columns_list.remove(x)
old_data_work_pivot = old_data.groupby(as_index=False, dropna=False, by=old_data_columns_list)[to_count].agg('sum')

In [132]:
# new_data_columns_list = ['companyCode', 'companyName', 'plantId', 'plantName','sector', 'sectorName', 'wbsElement', 'wbsElementName', 'vendorId','vendorName', 'projectTypeCode', 'projectTypeName', 'projectProfile','projectId', 'projectName', 'costElement', 'costElementName','versionCode', 'versionName', 'period', 'currency', 'amountIndicators']
# new_data_work = new_data[new_data_columns_list]
new_data_columns_list = ['companyCode', 'plantId', 'sector', 'wbsElement', 'vendorId', 'versionCode', 'period']
# to_remove = ['wbsElement','wbsElementName','projectTypeName','costElement','amountIndicators']
to_count = 'amountIndicators'
# for x in to_remove:
#     new_data_columns_list.remove(x)
new_data_work_pivot = new_data.groupby(as_index=False, dropna=False, by=new_data_columns_list)[to_count].agg('sum')

In [133]:
new_data_work_pivot['vendorId'], old_data_work_pivot['vendorId'] = new_data_work_pivot['vendorId'].astype(str), old_data_work_pivot['vendorId'].astype(str)
new_data_work_pivot['period'], old_data_work_pivot['calendarYearMonth'] = new_data_work_pivot['period'].astype(str), old_data_work_pivot['calendarYearMonth'].astype(str)
merge_data = old_data_work_pivot.merge(new_data_work_pivot, how='outer', left_on=old_data_columns_list, right_on=new_data_columns_list).sort_values(['calendarYearMonth', 'period'])

In [134]:
merge_data[(~merge_data.calendarYearMonth.isna())&(~merge_data.period.isna())]
# merge_data = is_approximately_equal_for_cols(merge_data, col_1='Сумма ПВК' ,col_2='amountIndicators', tolerance=0.4, print_diff=False, name_col_with_diff='')

,companyCode,plantId,sector,wbsElement,vendorId,versionCode,calendarYearMonth,Сумма ПВК,period,amountIndicators
1557,2100,2102,2A22,C.21020001-1.03.020,1000013981,35,10.2024,13921216.00,10.2024,13921216.00
1558,2100,2102,2A22,C.21020001-1.03.020,1000013981,45,10.2024,16705459.20,10.2024,16705459.20
1559,2100,2102,2A22,C.21020001-1.03.020,1000013981,75,10.2024,13921216.00,10.2024,13921216.00
1560,2100,2102,2A22,C.21020001-1.03.020,1000013981,85,10.2024,13921216.00,10.2024,13921216.00
1632,2100,2102,2A22,C.21020027-1.01.001,1000013981,45,10.2024,52961614.93,10.2024,50437906.68
...,...,...,...,...,...,...,...,...,...,...
102368,Y300,Y321,Y321,W.Y3210141-1.01.001.02,1000094921,85,12.2024,881362.00,12.2024,38333333.33
102587,Y300,Y321,Y321,W.Y3210178-1.01.001,1000086870,35,12.2024,4500000.00,12.2024,4500000.00
102588,Y300,Y321,Y321,W.Y3210178-1.01.001,1000086870,45,12.2024,3780000.00,12.2024,3780000.00
102589,Y300,Y321,Y321,W.Y3210178-1.01.001,1000086870,75,12.2024,3150000.00,12.2024,3150000.00


In [135]:
# to excel
Output_file = f'{str(date.today())}_ivest_wwwh.xlsx'
with pd.ExcelWriter(Output_file, engine='openpyxl') as writer:
    merge_data.to_excel(writer, index=False, sheet_name='merge')
    old_data.to_excel(writer, index=False, sheet_name='data_manual')
    new_data.to_excel(writer, index=False, sheet_name='data_wwwh')
